In [1]:
from openai import OpenAI

In [3]:
client = OpenAI()

Upload training file to OpenAI server

In [4]:
client.files.create(file=open("dataset/train.toon", 'rb'), purpose="fine-tune")

FileObject(id='file-56K1iy6m5MP9auqtFivMWQ', bytes=168281, created_at=1762311003, filename='train.toon', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

In [5]:
client.files.list()

SyncCursorPage[FileObject](data=[FileObject(id='file-56K1iy6m5MP9auqtFivMWQ', bytes=168281, created_at=1762311003, filename='train.toon', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)], has_more=False, object='list', first_id='file-56K1iy6m5MP9auqtFivMWQ', last_id='file-56K1iy6m5MP9auqtFivMWQ')

Retrieving a file

In [6]:
client.files.retrieve("file-56K1iy6m5MP9auqtFivMWQ") 

FileObject(id='file-56K1iy6m5MP9auqtFivMWQ', bytes=168281, created_at=1762311003, filename='train.toon', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

Creating a fine-tuning Job

Parameters:
model -> Pre-trained GPT model name which you want to fine-tune.

training-file -> ID of an uploaded file that contains training data in JSONL or Toon format.

hyperparameters -> An object containing the hyperparameters to be used in the fine-tuning job.

    - batch_size -> Set the number of examples in each batch. Defaults to auto.

    - learning_rate_multiplier -> Scaling factor for the learning rate. Defaults to auto.

    - n_epochs -> Refers to the number of epochs to train the model for. Defaults to auto.

Suffix -> A string of upto 18 characters to be added to fine-tuned name.

Seed -> An integer value used to control the reproducibility of the job.

Validation_file -> ID of an uploaded file that contains validation data in JSONL or Toon format.



In [8]:
client.fine_tuning.jobs.create(training_file="file-56K1iy6m5MP9auqtFivMWQ",
                               model="gpt-3.5-turbo",
                               hyperparameters={
                                   "n_epochs": 4
                               })

FineTuningJob(id='ftjob-Iau1uct3RB0Efmtg5NlP2xfw', created_at=1762311857, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=4), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-hbYKtyDbqYrnHlOyq41rI1AP', result_files=[], seed=1839880955, status='validating_files', trained_tokens=None, training_file='file-56K1iy6m5MP9auqtFivMWQ', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=4))), user_provided_suffix=None, usage_metrics=None, shared_with_openai=False, eval_id=None)

In [9]:
client.fine_tuning.jobs.list(limit=2)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-Iau1uct3RB0Efmtg5NlP2xfw', created_at=1762311857, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=4), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-hbYKtyDbqYrnHlOyq41rI1AP', result_files=[], seed=1839880955, status='validating_files', trained_tokens=None, training_file='file-56K1iy6m5MP9auqtFivMWQ', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=4))), user_provided_suffix=None, usage_metrics=None, shared_with_openai=False, eval_id=None)], has_more=False, object='list')

To retrieve the information about the job

In [19]:
status = client.fine_tuning.jobs.retrieve("ftjob-Iau1uct3RB0Efmtg5NlP2xfw")
print(status.status)
print("Your fine-tuned model ID:", status.fine_tuned_model)

running
Your fine-tuned model ID: None


To get the events that are running inside a job

In [ ]:
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-Iau1uct3RB0Efmtg5NlP2xfw", limit=5)


TypeError: Jobs.list_events() got an unexpected keyword argument 'stream'

Testing the fine-tuned model

In [ ]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "developer", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)
